In [2]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')
bus_capacity = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/refs/heads/main/datasets/bus_divisions_cleaned_capacity%20(1).csv')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Multi‐depot VRP: each route must start/end at a B-terminal,
visit all A and V nodes exactly once, and stay under MAX_ROUTE_TIME.
"""

import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# PARAMETERS
CSV_FILE = 'https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/new_matrix.csv'   # your matrix, with row/col headers = node IDs
MAX_ROUTE_TIME = 3 * 3600     # seconds (3 hours)

def main():
    # --- Load data ---
    df = pd.read_csv(CSV_FILE, index_col=0)
    nodes = list(df.index)
    time_matrix = df.values.tolist()

    # classify nodes
    terminals = [n for n in nodes if n.startswith('B')]
    accommodations = [n for n in nodes if n.startswith('A')]
    venues = [n for n in nodes if n.startswith('V')]

    # map node index ↔ ID
    node_to_idx = {n: i for i, n in enumerate(nodes)}

    # --- OR-Tools manager & model ---
    num_vehicles = len(terminals)
    starts = [node_to_idx[b] for b in terminals]
    ends = starts[:]  # start and end at same terminal
    manager = pywrapcp.RoutingIndexManager(len(nodes), num_vehicles, starts, ends)
    routing = pywrapcp.RoutingModel(manager)

    # --- Transit callback (travel times) ---
    def time_callback(from_index, to_index):
        i = manager.IndexToNode(from_index)
        j = manager.IndexToNode(to_index)
        return time_matrix[i][j]
    transit_cb_idx = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb_idx)

    # --- Time dimension & max-route-time constraint ---
    routing.AddDimension(
        transit_cb_idx,
        slack_max=0,               # no slack
        capacity=MAX_ROUTE_TIME,   # per‐route limit
        fix_start_cumul_to_zero=True,
        name='Time'
    )
    time_dim = routing.GetDimensionOrDie('Time')

    # --- Ensure every line includes at least one venue ---
    for veh_id in range(num_vehicles):
        venue_indices = [manager.NodeToIndex(node_to_idx[v]) for v in venues]
        routing.AddVariableMinimizedByFinalizer(
            routing.solver().Sum([routing.VehicleVar(idx) == veh_id for idx in venue_indices])
        )

    # --- Solve parameters ---
    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_params.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_params.time_limit.seconds = 30  # adjust if needed

    # --- Solve ---
    solution = routing.SolveWithParameters(search_params)
    if not solution:
        print("No solution found!")
        return []

    # --- Extract & return routes ---
    routes = []
    for veh_id in range(num_vehicles):
        index = routing.Start(veh_id)
        route_nodes = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route_nodes.append(nodes[node])
            index = solution.Value(routing.NextVar(index))
        # add final terminal
        end_node = manager.IndexToNode(index)
        route_nodes.append(nodes[end_node])

        # skip empty routes (start→end directly)
        if len(route_nodes) > 2:
            total_time = solution.Value(time_dim.CumulVar(routing.End(veh_id)))
            mins = total_time // 60
            print(f"Line {veh_id + 1} ({terminals[veh_id]}): {mins} minutes")
            routes.append(route_nodes)

    return routes

if __name__ == '__main__':
    main()

In [1]:
import folium
from folium import PolyLine, Marker
import random
import pandas as pd

def map_routes(accommodations_df, venues_df, terminals_df):
    # Get routes from the main function
    routes = main()
    if not routes:
        print("No routes to map.")
        return None

    # Create a base map centered around a default location
    base_map = folium.Map(location=[34.0522, -118.2437], zoom_start=10)  # Default: Los Angeles
    
    # Combine all coordinates into a single dictionary
    coordinates = {}
    for _, row in accommodations_df.iterrows():
        if not pd.isna(row['avg_latitude']) and not pd.isna(row['avg_longitude']):
            coordinates[row['id']] = (row['avg_latitude'], row['avg_longitude'])
    for _, row in venues_df.iterrows():
        if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
            coordinates[row['id']] = (row['Latitude'], row['Longitude'])
    for _, row in terminals_df.iterrows():
        if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
            coordinates[row['id']] = (row['Latitude'], row['Longitude'])

    # Add only nodes that are part of the routes to the map
    route_nodes = set(node for route in routes for node in route)
    for node in route_nodes:
        if node in coordinates:
            lat, lon = coordinates[node]
            if node.startswith('A'):
                Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='blue', icon='home')).add_to(base_map)
            elif node.startswith('V'):
                Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='red', icon='info-sign')).add_to(base_map)
            elif node.startswith('B'):
                Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='green', icon='cloud')).add_to(base_map)

    # Plot each route with a different color
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black']
    for i, route in enumerate(routes):
        route_coords = [coordinates[node] for node in route if node in coordinates]
        color = colors[i % len(colors)]  # Cycle through colors
        PolyLine(route_coords, color=color, weight=2.5, opacity=1).add_to(base_map)
    
    return base_map


accommodations_df = accomodations_clusters
venues_df = venues
terminals_df = bus_terminals
map_routes(accommodations_df, venues_df, terminals_df)

/Users/leonardogreco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'accomodations_clusters' is not defined

In [6]:
import pandas as pd
import numpy as np
import folium
from folium import PolyLine, Marker
from IPython.display import display

# ————— CONFIG —————
FILE             = "https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/new_matrix.csv"
EXCLUDED_DEPOTS  = {'BD16', 'BT24', 'BT08', 'BT16'}
MIN_ACC          = 2    # min A’s per route
MIN_VENUE        = 1    # min V’s per route

# ————— LOAD & DROP UNWANTED —————
df = pd.read_csv(FILE, index_col=0)
# drop both rows & cols for any excluded depots
df = df.drop(index=EXCLUDED_DEPOTS, columns=EXCLUDED_DEPOTS, errors='ignore')

ids = df.index.tolist()
D   = df.values.astype(int)   # seconds
n   = len(ids)

# classify indices
depots = [i for i,s in enumerate(ids) if s.startswith("B")]
accs   = {i for i,s in enumerate(ids) if s.startswith("A")}
vens   = {i for i,s in enumerate(ids) if s.startswith("V")}

# sanity checks
if len(depots) == 0:
    raise RuntimeError("No depots left after exclusion!")
if len(accs) < MIN_ACC * len(depots):
    raise RuntimeError(f"Need ≥{MIN_ACC*len(depots)} accommodations, got {len(accs)}")
if len(vens) < MIN_VENUE * len(depots):
    raise RuntimeError(f"Need ≥{MIN_VENUE*len(depots)} venues, got {len(vens)}")

# all other nodes to be assigned
nodes = set(range(n)) - set(depots)

# ————— SEED EACH ROUTE —————
routes = {d: [d] for d in depots}
acc_rem = set(accs)
ven_rem = set(vens)

for d in depots:
    # pick 2 nearest A’s
    for a in sorted(acc_rem, key=lambda x: D[d,x])[:MIN_ACC]:
        routes[d].append(a)
        acc_rem.remove(a)
    # pick 1 nearest V
    v = min(ven_rem, key=lambda x: D[d,x])
    routes[d].append(v)
    ven_rem.remove(v)

# ————— ASSIGN THE REST (round robin) —————
pool = list(acc_rem | ven_rem)
while pool:
    for d in depots:
        if not pool:
            break
        last = routes[d][-1]
        nxt  = min(pool, key=lambda x: D[last,x])
        routes[d].append(nxt)
        pool.remove(nxt)

# ————— FINISH & PRINT —————
coordinates = {}
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv').iterrows():
    if not pd.isna(row['avg_latitude']) and not pd.isna(row['avg_longitude']):
        coordinates[row['id']] = (row['avg_latitude'], row['avg_longitude'])
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv').iterrows():
    if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
        coordinates[row['id']] = (row['Latitude'], row['Longitude'])
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv').iterrows():
    if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
        coordinates[row['id']] = (row['Latitude'], row['Longitude'])

base_map = folium.Map(location=[34.0522, -118.2437], zoom_start=10)  # Default: Los Angeles
colors = ['blue', 'green', 'red', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black']

for d in depots:
    seq   = routes[d] + [d]
    t_sec = sum(D[seq[i], seq[i+1]] for i in range(len(seq)-1))
    stops = len(seq)-2
    names = " → ".join(ids[i] for i in seq)
    print(f"{ids[d]:<5}: {names}  ({stops} stops, {t_sec/60:.1f} min)")
    
    # Map the route
    route_coords = [coordinates[ids[node]] for node in seq if ids[node] in coordinates]
    color = colors[d % len(colors)]
    PolyLine(route_coords, color=color, weight=2.5, opacity=1).add_to(base_map)

# Add markers for all nodes
for node, (lat, lon) in coordinates.items():
    if node.startswith('A'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='blue', icon='home')).add_to(base_map)
    elif node.startswith('V'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='red', icon='info-sign')).add_to(base_map)
    elif node.startswith('B'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='green', icon='cloud')).add_to(base_map)

# Display the map
display(base_map)

BD14 : BD14 → A13 → A14 → V30 → A18 → V9 → V5 → BD14  (6 stops, 162.8 min)
BD15 : BD15 → A1 → A4 → V28 → A44 → A19 → A2 → BD15  (6 stops, 104.2 min)
BT03 : BT03 → A16 → A35 → V19 → A25 → A10 → A8 → BT03  (6 stops, 210.4 min)
BL14 : BL14 → A15 → A29 → V12 → A6 → A39 → A9 → BL14  (6 stops, 222.3 min)
BD04 : BD04 → A17 → A40 → V31 → A12 → A34 → A24 → BD04  (6 stops, 204.2 min)
BT11 : BT11 → A31 → A3 → V24 → V14 → A43 → A36 → BT11  (6 stops, 176.0 min)
BT06 : BT06 → A28 → A41 → V8 → A49 → A48 → A11 → BT06  (6 stops, 165.9 min)
BT21 : BT21 → A7 → A37 → V17 → A33 → A27 → A42 → BT21  (6 stops, 207.4 min)
BL20 : BL20 → A51 → A46 → V20 → A32 → A45 → A38 → BL20  (6 stops, 227.6 min)
BT18 : BT18 → A22 → A20 → V18 → A50 → A21 → A30 → BT18  (6 stops, 212.6 min)
BL23 : BL23 → A23 → A5 → V11 → A47 → A26 → BL23  (5 stops, 132.8 min)


In [8]:
# ————— 2-OPT LOCAL SEARCH —————
def route_distance(route, D):
    """Sum of D[i,j] over consecutive pairs in route."""
    return sum(D[route[i], route[i+1]] for i in range(len(route)-1))

def two_opt(route, D):
    """Improves route by swapping any i–j subsequence if it shortens the tour."""
    best = route[:]
    improved = True
    while improved:
        improved = False
        for i in range(1, len(best)-2):
            for j in range(i+1, len(best)-1):
                if j - i == 1: 
                    continue  # adjacent nodes, no real swap
                # build candidate by reversing the segment [i..j]
                cand = best[:i] + best[i:j+1][::-1] + best[j+1:]
                if route_distance(cand, D) < route_distance(best, D):
                    best = cand
                    improved = True
        # if we made any swap, loop again to try further improvements
    return best

# ————— APPLY 2-OPT & PRINT IMPROVED ROUTES —————
import folium
from folium import PolyLine, Marker
from IPython.display import display

coordinates = {}
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv').iterrows():
    if not pd.isna(row['avg_latitude']) and not pd.isna(row['avg_longitude']):
        coordinates[row['id']] = (row['avg_latitude'], row['avg_longitude'])
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv').iterrows():
    if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
        coordinates[row['id']] = (row['Latitude'], row['Longitude'])
for _, row in pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv').iterrows():
    if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']):
        coordinates[row['id']] = (row['Latitude'], row['Longitude'])

base_map = folium.Map(location=[34.0522, -118.2437], zoom_start=10)  # Default: Los Angeles
colors = ['blue', 'green', 'red', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black']

for depot_idx, seq in routes.items():
    # add return‐to‐depot at end
    initial = seq + [depot_idx]
    # Using the pre-defined cost matrix D from a previous cell
    before_sec = route_distance(initial, D)
    improved = two_opt(initial, D)
    after_sec  = route_distance(improved, D)

    # pretty‐print
    stops = len(improved) - 2
    names = " → ".join(ids[i] for i in improved)
    print(f"{ids[depot_idx]:<5}: {names}")
    print(f"       Stops: {stops}, Time: {after_sec/60:.1f} min "
          f"(was {before_sec/60:.1f} min)\n")
    
    # Map the improved route
    route_coords = [coordinates[ids[node]] for node in improved if ids[node] in coordinates]
    color = colors[depot_idx % len(colors)]
    PolyLine(route_coords, color=color, weight=2.5, opacity=1).add_to(base_map)

# Add markers for all nodes
for node, (lat, lon) in coordinates.items():
    if node.startswith('A'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='blue', icon='home')).add_to(base_map)
    elif node.startswith('V'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='red', icon='info-sign')).add_to(base_map)
    elif node.startswith('B'):
        Marker(location=(lat, lon), popup=node, icon=folium.Icon(color='green', icon='cloud')).add_to(base_map)

# Display the map
display(base_map)

BD14 : BD14 → A13 → V5 → A14 → V9 → A18 → V30 → BD14
       Stops: 6, Time: 135.7 min (was 162.8 min)

BD15 : BD15 → A1 → A19 → A44 → V28 → A4 → A2 → BD15
       Stops: 6, Time: 90.8 min (was 104.2 min)

BT03 : BT03 → A16 → A35 → A8 → A10 → A25 → V19 → BT03
       Stops: 6, Time: 179.4 min (was 210.4 min)

BL14 : BL14 → A15 → A9 → A29 → A39 → A6 → V12 → BL14
       Stops: 6, Time: 197.8 min (was 222.3 min)

BD04 : BD04 → V31 → A12 → A34 → A24 → A40 → A17 → BD04
       Stops: 6, Time: 172.7 min (was 204.2 min)

BT11 : BT11 → A3 → A36 → A43 → V24 → V14 → A31 → BT11
       Stops: 6, Time: 155.6 min (was 176.0 min)

BT06 : BT06 → A11 → A48 → A49 → V8 → A41 → A28 → BT06
       Stops: 6, Time: 161.6 min (was 165.9 min)

BT21 : BT21 → A7 → A37 → A27 → A33 → V17 → A42 → BT21
       Stops: 6, Time: 202.2 min (was 207.4 min)

BL20 : BL20 → A51 → A46 → A38 → A45 → A32 → V20 → BL20
       Stops: 6, Time: 224.7 min (was 227.6 min)

BT18 : BT18 → A21 → A30 → A50 → V18 → A20 → A22 → BT18
       Stops